In [1]:
import math
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt 
import pandas as pd
import pvlib
from pvlib.location import Location
%matplotlib notebook
# seaborn makes the plots look nicer
import seaborn as sns
sns.set_color_codes()

In [2]:
from pvlib.pvsystem import PVSystem
#from pvlib.location import Location
from pvlib.modelchain import ModelChain

# load some module and inverter specifications
sandia_modules = pvlib.pvsystem.retrieve_sam('SandiaMod')
cec_inverters = pvlib.pvsystem.retrieve_sam('cecinverter')
cec_modules = pvlib.pvsystem.retrieve_sam(name='CECMod')

In [3]:
module = sandia_modules['Canadian_Solar_CS5P_220M___2009_']
inverter = cec_inverters['ABB__MICRO_0_25_I_OUTD_US_208_208V__CEC_2014_']



In [4]:
module

Vintage                                                          2009
Area                                                            1.701
Material                                                         c-Si
Cells_in_Series                                                    96
Parallel_Strings                                                    1
Isco                                                          5.09115
Voco                                                          59.2608
Impo                                                          4.54629
Vmpo                                                          48.3156
Aisc                                                         0.000397
Aimp                                                         0.000181
C0                                                            1.01284
C1                                                         -0.0128398
Bvoco                                                        -0.21696
Mbvoc               

In [5]:
inverter

Vac          208.000000
Paco         250.000000
Pdco         259.522050
Vdco          40.242603
Pso            1.771614
C0            -0.000025
C1            -0.000090
C2             0.000669
C3            -0.018900
Pnt            0.020000
Vdcmax        65.000000
Idcmax        10.000000
Mppt_low      20.000000
Mppt_high     50.000000
Name: ABB__MICRO_0_25_I_OUTD_US_208_208V__CEC_2014_, dtype: float64

In [7]:



system = {'module': module, 'inverter': inverter,
           'surface_azimuth': 180}






In [8]:
system

{'inverter': Vac          208.000000
 Paco         250.000000
 Pdco         259.522050
 Vdco          40.242603
 Pso            1.771614
 C0            -0.000025
 C1            -0.000090
 C2             0.000669
 C3            -0.018900
 Pnt            0.020000
 Vdcmax        65.000000
 Idcmax        10.000000
 Mppt_low      20.000000
 Mppt_high     50.000000
 Name: ABB__MICRO_0_25_I_OUTD_US_208_208V__CEC_2014_, dtype: float64,
 'module': Vintage                                                          2009
 Area                                                            1.701
 Material                                                         c-Si
 Cells_in_Series                                                    96
 Parallel_Strings                                                    1
 Isco                                                          5.09115
 Voco                                                          59.2608
 Impo                                                        

In [9]:
coordinates=[(53.03, -112.49, 'Camrose', 739, 'Etc/GMT+7'),
                  (53.55, -114.11, 'Edmonton', 766, 'Etc/GMT+7'),
                  (55.18, -118.88, 'GrandPrairie', 669 , 'Etc/GMT+7'), 
             (49.68, -112.75, 'Lethebridge', 903, 'Etc/GMT+7'),
                (56.23,-117.45, 'PeaceRiver', 570, 'Etc/GMT+7'),
             (49.52,-114.00, 'PincherCreek', 1189.6, 'Etc/GMT+7'),          
               (51.05, -114.0852, 'Calgary', 1042,'Etc/GMT+7')
            ]

In [10]:
latitude, longitude, name, altitude, timezone = coordinates[0]

In [11]:
chosen6 = pd.read_csv("chosen6solar.csv")

In [12]:
chosen6[["name",'tz','Latitude','Longitude','Elevation']]

,name,tz,Latitude,Longitude,Elevation
0,CAMROSE,Etc/GMT+7,53.05,-112.82,739.0
1,EDMONTON-STONY-PLAIN-CS,Etc/GMT+7,53.55,-114.11,766.0
2,GRANDE-PRAIRIE-A,Etc/GMT+7,55.18,-118.88,669.0
3,LETHBRIDGE-DEMO-FARM-AGDM,Etc/GMT+7,49.68,-112.75,903.0
4,PEACE-RIVER-A,Etc/GMT+7,56.23,-117.45,570.9
5,PINCHER-CREEK,Etc/GMT+7,49.52,-114.00,1189.6


In [13]:
chosen6[["name",'tz','Latitude','Longitude','Elevation']].iloc[1,:]

name         EDMONTON-STONY-PLAIN-CS
tz                         Etc/GMT+7
Latitude                       53.55
Longitude                    -114.11
Elevation                        766
Name: 1, dtype: object

In [14]:
def convert_to_ac(i):
    
    name, timezone, latitude, longitude, altitude = chosen6[["name",'tz','Latitude','Longitude','Elevation']].iloc[i,:]
    
    solar_plant = pd.read_csv("chosen_stations/"+ name + ".csv",low_memory=False)

    solar_plant.rename(columns={'y': 'year', 'm': 'month', 'd': 'day', 'h':'hour'}, inplace=True)

    solar_plant["times"]  =pd.to_datetime(solar_plant[['year', 'month', 'day', 'hour']])

    times = pd.DatetimeIndex(solar_plant["times"]).tz_localize(timezone)

    solar_plant["times"] =times

    solar_plant.set_index(pd.DatetimeIndex(solar_plant["times"]), inplace= True)

    wind_speed = solar_plant.winS/10

    temp_air = solar_plant["tem"]/10

    system['surface_tilt'] = latitude

    solpos = pvlib.solarposition.get_solarposition(times, latitude, longitude)

    #dni_extra = pvlib.irradiance.extraradiation(times)

    dni_extra=solar_plant.ir*1000/3600

    dni_extra = pd.Series(dni_extra, index=times)


    airmass = pvlib.atmosphere.relativeairmass(solpos['apparent_zenith'])
    # pressure = pvlib.atmosphere.alt2pres(altitude)
    # use historical pressure
    pressure = solar_plant.pre*10

    am_abs = pvlib.atmosphere.absoluteairmass(airmass, pressure)

    tl = pvlib.clearsky.lookup_linke_turbidity(times, latitude, longitude)

    # use our data from CWEEC
    
    # cs = pvlib.clearsky.ineichen(solpos['apparent_zenith'], am_abs, tl,
    #                              dni_extra=dni_extra, altitude=altitude)
    cs = pd.DataFrame()
    cs['dni'], cs['ghi'], cs['dhi']= solar_plant.dni*1000/3600, solar_plant.ghi*1000/3600,solar_plant.dhi*1000/3600

    aoi = pvlib.irradiance.aoi(system['surface_tilt'], system['surface_azimuth'],
                               solpos['apparent_zenith'], solpos['azimuth'])

    total_irrad = pvlib.irradiance.total_irrad(system['surface_tilt'],
                                               system['surface_azimuth'],
                                               solpos['apparent_zenith'],
                                               solpos['azimuth'],
                                               cs['dni'], cs['ghi'], cs['dhi'],
                                               dni_extra=dni_extra,
                                               model='haydavies')

    temps = pvlib.pvsystem.sapm_celltemp(total_irrad['poa_global'],
                                         wind_speed, temp_air)

    effective_irradiance = pvlib.pvsystem.sapm_effective_irradiance(
        total_irrad['poa_direct'], total_irrad['poa_diffuse'],
        am_abs, aoi, module)

    dc = pvlib.pvsystem.sapm(effective_irradiance, temps['temp_cell'], module)
    ac = pvlib.pvsystem.snlinverter(dc['v_mp'], dc['p_mp'], inverter)
    #annual_energy = ac.sum()

    solar_plant['ac']=ac
    solar_plant.to_csv( name + '_ac.csv')

In [18]:
# W/hr
convert_to_ac(5)


/home/jon/anaconda2/envs/py36/lib/python3.6/site-packages/pvlib/pvsystem.py:1303: RuntimeWarning: divide by zero encountered in log
  module['Voco'] + module['Cells_in_Series']*delta*np.log(Ee) +
/home/jon/anaconda2/envs/py36/lib/python3.6/site-packages/pvlib/pvsystem.py:1309: RuntimeWarning: divide by zero encountered in log
  module['C3']*module['Cells_in_Series']*((delta*np.log(Ee)) ** 2) +
